In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import dvc.api
import pandas as pd
import matplotlib as plt

In [3]:
sys.path.append('.')
sys.path.insert(1, '../script/')
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import warnings
import sys
import datetime

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

In [9]:
data_file = '/train_data.csv'

# the local data path
local_path = '../data/' + data_file

# the path to the data set
path = 'data/' + data_file

# base repository
repo = '../'

In [10]:
version = 'v5'

# data path using dvc api
data_url = dvc.api.get_url(path = path, repo = repo, rev = version)

# reading the csv file
missing_values = ["n/a", "na", "undefined", '?', 'NA', 'undefined']
train_data = pd.read_csv(data_url, na_values=missing_values)
train_data.head()

,Date,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day
0,2015-07-31,1,c,a,20.0,9.0,2008.0,0,13.0,2010.0,...,5,5263,555,1,1,0,1,2015,7,31
1,2015-07-30,1,c,a,20.0,9.0,2008.0,0,13.0,2010.0,...,4,5020,546,1,1,0,1,2015,7,30
2,2015-07-29,1,c,a,20.0,9.0,2008.0,0,13.0,2010.0,...,3,4782,523,1,1,0,1,2015,7,29
3,2015-07-28,1,c,a,20.0,9.0,2008.0,0,13.0,2010.0,...,2,5011,560,1,1,0,1,2015,7,28
4,2015-07-27,1,c,a,20.0,9.0,2008.0,0,13.0,2010.0,...,1,6102,612,1,1,0,1,2015,7,27


we have to remove the column where the store is not open and the sales value is 0, since it has no effect.

In [11]:
train_data = train_data[train_data['Open'] == 1]
train_data = train_data[train_data['Sales'] > 0.0]

Then we select the prediction variable and the targe variable as y and x respectively.

In [12]:
# predicted value
y = train_data['Sales']

X = train_data.drop('Sales',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

Here we use sklearn pipeline

In [13]:
data_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])

pipeline = Pipeline(steps = [
               ('preprocessor', data_transformer)
              ,('regressor',RandomForestRegressor(max_depth=64, random_state=0))
           ])

In [24]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '2013-12-05'